# IMP Logics DEMO

## LOADING PROJECT DEPENDENCIES

In [1]:
%%loadFromPOM
<dependencies>
    <!-- ANTLR - should actually not be needed, problem with imp-logics.jar -->
    <dependency>
        <groupId>org.antlr</groupId>
        <artifactId>antlr4-runtime</artifactId>
        <version>4.12.0</version>
    </dependency>
    <!-- ASSERTJ - for testings and asserts -->
    <dependency>
        <groupId>org.assertj</groupId>
        <artifactId>assertj-core</artifactId>
        <version>3.24.2</version>
        <scope>test</scope>
    </dependency>
</dependencies>

In [2]:
%maven org.assertj:assertj-core:3.24.2
%jars imp-logics-2.0.0-SNAPSHOT.jar
%jars imp-logics-2.0.0-SNAPSHOT-tests.jar
%jars ontological-queries-rewriting-1.0-SNAPSHOT.jar

import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.domain.DependencySchema;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.domain.TGD;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.services.analyzers.DatalogPlusMinusAnalyzer;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.services.analyzers.egds.NonConflictingEGDsAnalyzer;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.services.parser.DependencySchemaParser;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.services.printer.DependencySchemaPrinter;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.assertions.LogicSchemaAssertions;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.domain.*;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.parser.LogicSchemaWithIDsParser;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.parser.QueryParser;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.printer.LogicSchemaPrinter;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.printer.QueryPrinter;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.processes.EqualityReplacer;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.processes.LogicProcessPipeline;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.processes.SchemaUnfolder;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.processes.SingleDerivationRuleTransformer;
import edu.upc.fib.inlab.imp.kse.ontological_queries_rewriting.OBDAMapping;
import edu.upc.fib.inlab.imp.kse.ontological_queries_rewriting.Rewriter;
import edu.upc.fib.inlab.imp.kse.ontological_queries_rewriting.utils.normalizers.TGDNormalizerProcess;

import java.util.Collection;
import java.util.HashSet;
import java.util.List;
import java.util.Set;

import static org.assertj.core.api.Assertions.assertThat;

In [3]:
private static void printWithHeader(String logicConstraintUsedVariables, String content) {
    System.out.println("\u001B[1m" + logicConstraintUsedVariables + ":\033[0m ");
    System.out.println(content);
}

private static void printWithHeaderInline(String logicConstraintUsedVariables, String content) {
    System.out.print("\u001B[1m" + logicConstraintUsedVariables + ":\033[0m ");
    System.out.println(content);
}
private static void printHeaderInline(String logicConstraintUsedVariables) {
    System.out.print("\u001B[1m" + logicConstraintUsedVariables + ":\033[0m ");
}

## DEMO START

### LOADING LOGIC SCHEMA UTILS & TOOLS

In [4]:
LogicSchemaWithIDsParser logicSchemaParser = new LogicSchemaWithIDsParser();
LogicSchemaPrinter logicSchemaPrinter = new LogicSchemaPrinter();
QueryParser queryParser = new QueryParser();
QueryPrinter queryPrinter = new QueryPrinter();

### PARSING & PRINTING LOGIC SCHEMA (CONTAINS LOGIC CONSTRAINTS AND MAPPINGS)

In [5]:
String logicSchemaString = """
    %% Schema Logic Constraints
    % AcademicRecord reference key to Student
    @AcademicRecordFKToStudent :- DB_AcademicRecord(studentName, subject, eval), not(IsStudent(studentName))
    IsStudent(studentName) :- DB_Student(studentName, age)
                  
    % Teacher must be over 18
    @TeachersMustBeOver18 :- Teacher_view(name, age), age < 18
    Teacher_view(name, age) :- DB_AssistantTeacher(name, age)
    Teacher_view(name, age) :- DB_TenuredTeacher(name, age)
                    
    % A teacher cannot teach himself
    @TeacherCannotTeachHimself :- DB_Teaches(teacherName, subject), DB_Studies(studentName, subject), teacherName=studentName
    """;
Set<Predicate> extraPredicates = Set.of(
    new Predicate("DB_ComposesPlan", 2),
    new Predicate("DB_PublishesAbout", 3)
);
LogicSchema logicSchema = logicSchemaParser.parse(logicSchemaString, extraPredicates);
Set<Predicate> logicSchemaPredicates = logicSchema.getAllPredicates();

printWithHeader("Logic Schema", logicSchemaPrinter.print(logicSchema));
printHeaderInline("Predicates");
for (Predicate p : logicSchemaPredicates) System.out.print(logicSchemaPrinter.visit(p) + " ");

Logic Schema: 
@TeacherCannotTeachHimself :- DB_Teaches(teacherName, subject), DB_Studies(studentName, subject), teacherName=studentName
@AcademicRecordFKToStudent :- DB_AcademicRecord(studentName, subject, eval), not(IsStudent(studentName))
@TeachersMustBeOver18 :- Teacher_view(name, age), age<18
IsStudent(studentName) :- DB_Student(studentName, age)
Teacher_view(name, age) :- DB_AssistantTeacher(name, age)
Teacher_view(name, age) :- DB_TenuredTeacher(name, age)

Predicates: IsStudent DB_PublishesAbout DB_Teaches DB_Student DB_TenuredTeacher DB_AssistantTeacher Teacher_view DB_Studies DB_ComposesPlan DB_AcademicRecord 

### LOGIC SCHEMA OBJECTS MANIPULATION
We will focus on constraint `@AcademicRecordFKToStudent`

In [6]:
LogicConstraint selectedConstraint = logicSchema.getLogicConstraintByID(new ConstraintID("AcademicRecordFKToStudent"));
printWithHeaderInline("Selected Logic Constraint", logicSchemaPrinter.visit(selectedConstraint));

Selected Logic Constraint: @AcademicRecordFKToStudent :- DB_AcademicRecord(studentName, subject, eval), not(IsStudent(studentName))


We can check the used variables in the constraint body

In [7]:
Set<Variable> usedVariables = selectedConstraint.getBody().getUsedVariables();
printHeaderInline("Used Variables in Body");
for (Variable v : usedVariables) System.out.print(logicSchemaPrinter.visit(v) + " ");

Used Variables in Body: studentName subject eval 

We can select a literal of the constraint

In [8]:
OrdinaryLiteral olit = (OrdinaryLiteral) selectedConstraint.getBody().get(1);
printWithHeaderInline("Selected Ordinary Literal", logicSchemaPrinter.visit(olit));

System.out.print("Ordinary Literal is negative: " + olit.isNegative());
assertThat(olit.isNegative()).isTrue();

Selected Ordinary Literal: not(IsStudent(studentName))
Ordinary Literal is negative: true

org.assertj.core.api.BooleanAssert@1

The predicate of an ordinary literal can be obtained

In [9]:
Predicate olitPredicate = olit.getPredicate();
printWithHeaderInline("Obtained Predicate", logicSchemaPrinter.visit(olitPredicate));
System.out.println("Predicate Literal is base: " + olitPredicate.isBase());
assertThat(olitPredicate.isBase()).isFalse();
System.out.println("Predicate Literal is derived: " + olitPredicate.isDerived());
assertThat(olitPredicate.isDerived()).isTrue();

Obtained Predicate: IsStudent
Predicate Literal is base: false
Predicate Literal is derived: true


org.assertj.core.api.BooleanAssert@1

From a derived predicate we can access it's definition rules

In [10]:
List<DerivationRule> derivationRules = olitPredicate.getDerivationRules();
printHeaderInline("Predicate's derivation rules:");
for (DerivationRule dr : derivationRules) System.out.print(logicSchemaPrinter.visit(dr));

Predicate's derivation rules:: IsStudent(studentName) :- DB_Student(studentName, age)

### LOGIC SCHEMA PROPERTY CHECKING

Other schema property checks can be performed

In [11]:
System.out.println("Selected Constraint(" + selectedConstraint.getID() + ") is safe: " + selectedConstraint.isSafe());
assertThat(selectedConstraint.isSafe()).isTrue();
System.out.println("Selected Ordinary Literal(" + logicSchemaPrinter.visit(olit) + ") is ground: " + olit.isGround());
assertThat(olit.isGround()).isFalse();

Selected Constraint(AcademicRecordFKToStudent) is safe: true
Selected Ordinary Literal(not(IsStudent(studentName))) is ground: false


org.assertj.core.api.BooleanAssert@1

### LOGIC SCHEMA TRANSFORMATIONS

Now we will apply a transformation pipeline over the original schema,

precisely, applying the `EqualityReplacer`, `SchemaUnfolder` & `SingleDerivationRuleTransformer` processes

In [12]:
LogicProcessPipeline pipeline = new LogicProcessPipeline(List.of(
        new EqualityReplacer(),
        new SchemaUnfolder(false),
        new SingleDerivationRuleTransformer()
));
LogicSchema modifiedLogicSchema = pipeline.execute(logicSchema);
printWithHeader("Modified schema", logicSchemaPrinter.print(modifiedLogicSchema));

Modified schema: 
@TeacherCannotTeachHimself :- DB_Teaches(teacherName, subject), DB_Studies(teacherName, subject)
@AcademicRecordFKToStudent :- DB_AcademicRecord(studentName, subject, eval), not(IsStudent(studentName))
@TeachersMustBeOver18_1 :- DB_AssistantTeacher(name, age), age<18
@TeachersMustBeOver18_2 :- DB_TenuredTeacher(name, age), age<18
IsStudent(studentName) :- DB_Student(studentName, age)
Teacher_view_2(name, age) :- DB_TenuredTeacher(name, age)
Teacher_view_1(name, age) :- DB_AssistantTeacher(name, age)



### LOADING DEPENDENCY SCHEMA UTILS & TOOLS

In [13]:
DependencySchemaParser dependencySchemaParser = new DependencySchemaParser();
DependencySchemaPrinter dependencySchemaPrinter = new DependencySchemaPrinter();

### PARSING & PRINTING DEPENDENCY SCHEMA

In [14]:
String dependencySchemaString = """
    % If a student passes a subject, the student has some evaluation
    HasPassed(student, subject) -> Exam(teacher, student, subject, data)
                    
    % If a teacher teaches a subject a student is coursing, the teacher evaluates the student
    Teaches(teacher, subject), Studies(student, subject) -> Exam(teacher, student, subject, data)
                    
    % If a teacher is expert in a subject from a study plan, the teacher gives the subject
    ExpertIn(teacher, subject), ComposesPlan(subject, studyPlan) -> Teaches(teacher, subject)
                    
    % A subject has, at most, one teacher
    % Teaches(teacher1, subject), Teaches(teacher2, subject) -> teacher1=teacher2
    """;
DependencySchema dependencySchema = dependencySchemaParser.parse(dependencySchemaString);
Set<Predicate> dependencySchemaPredicates = logicSchema.getAllPredicates();

printWithHeader("Dependency Schema", dependencySchemaPrinter.print(dependencySchema));
printHeaderInline("Predicates");
for (Predicate p : dependencySchemaPredicates) System.out.print(logicSchemaPrinter.visit(p) + " ");

Dependency Schema: 
HasPassed(student, subject) -> Exam(teacher, student, subject, data)
Teaches(teacher, subject), Studies(student, subject) -> Exam(teacher, student, subject, data)
ExpertIn(teacher, subject), ComposesPlan(subject, studyPlan) -> Teaches(teacher, subject)

Predicates: IsStudent DB_PublishesAbout DB_Teaches DB_Student DB_TenuredTeacher DB_AssistantTeacher Teacher_view DB_Studies DB_ComposesPlan DB_AcademicRecord 

### DEPENDENCY PROPERTY CHECKING

In [15]:
TGD tgd = dependencySchema.getAllTGDs().get(0);
printWithHeaderInline("Selected TGD", dependencySchemaPrinter.visit(tgd));

System.out.println("TGD is linear: " + tgd.isLinear());
assertThat(tgd.isLinear()).isTrue();
System.out.println("TGD is guarded: " + tgd.isGuarded());
assertThat(tgd.isGuarded()).isTrue();

Selected TGD: HasPassed(student, subject) -> Exam(teacher, student, subject, data)
TGD is linear: true
TGD is guarded: true


org.assertj.core.api.BooleanAssert@1

### DEPENDENCY SCHEMA PROPERTY CHECKING

First, we will check that no EGD is conflicting with the TGDs in the dependency schema

In [16]:
NonConflictingEGDsAnalyzer nonConflictingEGDsAnalyzer = new NonConflictingEGDsAnalyzer();
boolean separable = nonConflictingEGDsAnalyzer.areEGDsNonConflictingWithTGDs(dependencySchema);
System.out.print("EGDs of schema are non conflicting / separable: " + separable);
assertThat(separable).isTrue();

EGDs of schema are non conflicting / separable: true

org.assertj.core.api.BooleanAssert@1

We will now analyze which Datalog+/- languages this dependency schema satisfies

In [17]:
DatalogPlusMinusAnalyzer analyzer = new DatalogPlusMinusAnalyzer();
Set<DatalogPlusMinusAnalyzer.DatalogPlusMinusLanguage> languages = analyzer.getDatalogPlusMinusLanguages(dependencySchema);
printHeaderInline("This dependency schema is: ");
for (DatalogPlusMinusAnalyzer.DatalogPlusMinusLanguage dl : languages) System.out.print(dl.name() + " ");

This dependency schema is: : WEAKLY_GUARDED STICKY 

### REWRITING PROCESS

Now let's perform a rewriting of an ontological query.

The loaded Logic Schema describes de relational database with with its Logic Constraints and Predicates.

The loaded Dependency Schema describes de ontology with its TGDs and ontological predicates.

Now, we will parse and load the mapping which links the ontology and relational database as well as the to be rewritten ontological query.

The TGDs need to be normalized before the rewriting process. We normalize the dependencySchema.

In [22]:
DependencySchema normalizedDependencySchema = new TGDNormalizerProcess().normalize(dependencySchema);
Set<Predicate> normalizedDependencySchemaPredicates = normalizedDependencySchema.getAllPredicates();

printWithHeader("Normalized Dependency Schema", dependencySchemaPrinter.print(normalizedDependencySchema));
printHeaderInline("Predicates");
for (Predicate p : normalizedDependencySchemaPredicates) System.out.print(logicSchemaPrinter.visit(p) + " ");

Normalized Dependency Schema: 
HasPassed(student, subject) -> AUX1(student, subject, teacher)
AUX1(student, subject, teacher) -> AUX2(student, subject, teacher, data)
AUX2(student, subject, teacher, data) -> Exam(teacher, student, subject, data)
Teaches(teacher, subject), Studies(student, subject) -> Exam(teacher, student, subject, data)
ExpertIn(teacher, subject), ComposesPlan(subject, studyPlan) -> Teaches(teacher, subject)

Predicates: AUX2 Studies AUX1 HasPassed Teaches ComposesPlan Exam ExpertIn 

The loaded Logic Schema will provide the mappings between the ontology and the relational tables

In [44]:
String mappingDBQueriesString = """
    % HasPassed(student, subject)
    (student, subject) :- DB_AcademicRecord(student, subject, mark), mark > 5
    
    % Exam(teacher, student, subject, data)
    (teacher, student, subject, data) :- DB_Exam(teacher, student, subject, data)
    
    % Teaches(teacher, subject)
    (teacher, subject) :- DB_Teaches(teacher, subject)
    
    % Studies(student, subject)
    (student, subject) :- DB_Studies(student, subject)
    
    % ExpertIn(teacher, subject)
    (teacher, subject) :- DB_PublishesAbout(teacher, paper, subject), DB_PublishesAbout(teacher, paper2, subject), paper<>paper2
    
    % ComposesPlan(subject, studyPlan)
    (subject, studyPlan) :- DB_ComposesPlan(subject, studyPlan)
    """;
List<Query> mappingDBQueries = queryParser.parse(mappingDBQueriesString, logicSchemaPredicates);
OBDAMapping mapping = new OBDAMapping.OBDAMappingBuilder()
    .addMapping(normalizedDependencySchema.getPredicateByName("HasPassed"), mappingDBQueries.get(0))
    .addMapping(normalizedDependencySchema.getPredicateByName("Exam"), mappingDBQueries.get(1))
    .addMapping(normalizedDependencySchema.getPredicateByName("Teaches"), mappingDBQueries.get(2))
    .addMapping(normalizedDependencySchema.getPredicateByName("Studies"), mappingDBQueries.get(3))
    .addMapping(normalizedDependencySchema.getPredicateByName("ExpertIn"), mappingDBQueries.get(4))
    .addMapping(normalizedDependencySchema.getPredicateByName("ComposesPlan"), mappingDBQueries.get(5))
    .build();

Let's parse the ontological query

In [45]:
String queryString = """
    % Ontological Query
    (student) :- Exam(teacher, student, subject, data)
    """;
Query query = queryParser.parse(queryString, normalizedDependencySchemaPredicates).get(0);
assertThat(query.isConjunctiveQuery()).isTrue();
ConjunctiveQuery ontologicalQuery = (ConjunctiveQuery) query;

### APPLYING REWRITING PROCEDURE

In [46]:
Set<TGD> ontologyTGDs = new HashSet<>(normalizedDependencySchema.getAllTGDs());
List<ConjunctiveQuery> rewriting = Rewriter.rewrite(ontologicalQuery, ontologyTGDs);

### PRINTING PARTIAL REWRITING

In [47]:
for (int i = 0; i < rewriting.size(); i++) {
    Query queryToPrint = rewriting.get(i);
    printWithHeader("Query " + i, queryPrinter.print(queryToPrint));
}

Query 0: 
(student) :- Exam(teacher, student, subject, data)
Query 1: 
(student) :- Teaches(teacher, subject), Studies(student, subject)
Query 2: 
(student) :- AUX2(student, subject, teacher, data)
Query 3: 
(student) :- ExpertIn(teacher, subject), ComposesPlan(subject, unfTGD0), Studies(student, subject)
Query 4: 
(student) :- AUX1(student, subject, teacher)
Query 5: 
(student) :- HasPassed(student, subject)


### APPLYING MAPPINGS

In [48]:
List<Query> finalRewriting = rewriting.stream()
    .map(mapping::translateToDBQueries)
    .flatMap(Collection::stream)
    .toList();

### PRINTING REWRITING

In [49]:
for (int i = 0; i < finalRewriting.size(); i++) {
    Query queryToPrint = finalRewriting.get(i);
    printWithHeader("Query " + i, queryPrinter.print(queryToPrint));
}

Query 0: 
(student) :- DB_Exam(teacher, student, subject, data)
Query 1: 
(student) :- DB_Teaches(teacher, subject), DB_Studies(student, subject)
Query 2: 
(student) :- DB_PublishesAbout(teacher, paper, subject), DB_PublishesAbout(teacher, paper2, subject), paper<>paper2, DB_ComposesPlan(subject, unfTGD0), DB_Studies(student, subject)
Query 3: 
(student) :- DB_AcademicRecord(student, subject, mark), mark>5


### ASSERT REWRITING IS EXPECTED

In [50]:
String expectedRewritingString = """
    % Expected Rewriting
    (student) :- DB_Exam(teacher, student, subject, data)
    (student) :- DB_AcademicRecord(student, subject, mark), mark > 5
    (student) :- DB_Teaches(teacher, subject), DB_Studies(student, subject)
    (student) :- DB_PublishesAbout(teacher, paper, subject), DB_PublishesAbout(teacher, paper2, subject), paper<>paper2, DB_ComposesPlan(subject, studyPlan), DB_Studies(student, subject)
    """;
List<Query> expectedRewriting = queryParser.parse(expectedRewritingString, normalizedDependencySchemaPredicates);
assertThat(finalRewriting)
    .satisfiesOnlyOnce(q -> LogicSchemaAssertions.assertThat(q).isIsomorphicTo(expectedRewriting.get(0)))
    .satisfiesOnlyOnce(q -> LogicSchemaAssertions.assertThat(q).isIsomorphicTo(expectedRewriting.get(1)))
    .satisfiesOnlyOnce(q -> LogicSchemaAssertions.assertThat(q).isIsomorphicTo(expectedRewriting.get(2)))
    .satisfiesOnlyOnce(q -> LogicSchemaAssertions.assertThat(q).isIsomorphicTo(expectedRewriting.get(3)));

org.assertj.core.api.ListAssert@1

## DEMO END